In [128]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import matplotlib.animation as animation

In [129]:
omega = 5.0         

In [130]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(None, 2)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(2)
])

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [131]:
def regression_accuracy(y_true, y_pred, tolerance=0.1):
    return tf.reduce_mean(tf.cast(tf.less_equal(tf.abs(y_true - y_pred), tolerance), tf.float32))

In [132]:
model.compile(optimizer='adam', loss='mse', metrics=[regression_accuracy])

In [133]:
def harmonic_oscillator(t, y):
    x, v = y
    dx_dt = v
    dv_dt = -omega**2 * x
    return [dx_dt, dv_dt]

In [134]:
initial_conditions = [1.0, 0.0]  
sol = solve_ivp(harmonic_oscillator, [0, 10], initial_conditions, t_eval=np.linspace(0, 10, 110))

In [135]:
X_train = []
Y_train = []

In [136]:
for i in range(len(sol.t) - 10):
    X_train.append(sol.y[:, i:i+10].T)
    Y_train.append(sol.y[:, i+10])

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [137]:
history = model.fit(X_train, Y_train, epochs=100, callbacks=[tf.keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs: print(f"Epoch {epoch+1} - Loss: {logs['loss']:.4f}, Regression Accuracy: {logs['regression_accuracy']:.4f}"))
])

Epoch 1/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 6.5718 - regression_accuracy: 0.0156Epoch 1 - Loss: 5.9240, Regression Accuracy: 0.0273
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 6.1972 - regression_accuracy: 0.0307
Epoch 2/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.8891 - regression_accuracy: 0.0625Epoch 2 - Loss: 4.5486, Regression Accuracy: 0.0586
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.9023 - regression_accuracy: 0.0510 
Epoch 3/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 3.9315 - regression_accuracy: 0.0469Epoch 3 - Loss: 3.0559, Regression Accuracy: 0.0391
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.3206 - regression_accuracy: 0.0432 
Epoch 4/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.7138 - regression_accuracy: 0.0156Epoch 4 - Loss: 1.5686, Regression Accuracy: 0.0352
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.6184 - regression_accuracy: 0.0297 
Epoch 5/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.7348 - regression_accuracy:

In [138]:
initial_conditions_test = sol.y[:, :10].T

In [139]:
predictions = []

In [140]:
predictions = []
for _ in range(100):
    # Predict using the reshaped initial conditions
    prediction = model.predict(initial_conditions_test.reshape(1, 10, 2))
    predictions.append(prediction[0])
    
    # Update the initial conditions for the next prediction
    initial_conditions_test = np.roll(initial_conditions_test, -1, axis=0)
    initial_conditions_test[-1, :] = prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━

In [141]:
predictions = np.array(predictions)
true_states = sol.y[:, 10:110].T

In [142]:
time = np.arange(len(predictions))

In [143]:
import matplotlib.pyplot as plt

# Plot training history and save it
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Loss')
if 'regression_accuracy' in history.history:
    plt.plot(history.history['regression_accuracy'], label='Regression Accuracy')
plt.title('Training Progress')
plt.xlabel('Epoch')
plt.legend()
plt.savefig('training_history.png')  # Save the figure as a PNG file
plt.close()  # Close the plot to free up memory

# Plot predictions vs. true values and save it
fig, axs = plt.subplots(2, 1, figsize=(12, 10))
axs[0].plot(time, predictions[:, 0], 'r-', label='Predicted Position')
axs[0].plot(time, true_states[:, 0], 'b--', label='True Position')
axs[0].set_title('Position Over Time')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Position')
axs[0].legend()

axs[1].plot(time, predictions[:, 1], 'r-', label='Predicted Velocity')
axs[1].plot(time, true_states[:, 1], 'b--', label='True Velocity')
axs[1].set_title('Velocity Over Time')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Velocity')
axs[1].legend()

plt.tight_layout()
plt.savefig('predictions_vs_true_values.png')  # Save the figure as a PNG file
plt.close()  # Close the plot to free up memory


In [144]:
test_data = sol.y[:, -120:]
initial_conditions_test = test_data[:, :10].T
X_test = []
Y_test = []

for i in range(test_data.shape[1] - 10):
    X_test.append(test_data[:, i:i+10].T)
    Y_test.append(test_data[:, i+10])

X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [145]:
# Evaluate the model on the test data
test_predictions = model.predict(X_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [146]:
# Evaluate the model on the test data
loss, reg_accuracy = model.evaluate(X_test, Y_test, verbose=0)

# Print the results
print(f'Test Loss: {loss:.4f}')
print(f'Test Regression Accuracy: {reg_accuracy:.4f}')

Test Loss: 0.0001
Test Regression Accuracy: 1.0000


In [147]:
test_predictions = model.predict(X_test)
time_test = np.arange(len(Y_test))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [148]:
# Assuming you have 'time_test', 'Y_test', and 'test_predictions' available
fig, axs = plt.subplots(2, 1, figsize=(12, 10))

# Plotting the true and predicted positions
axs[0].plot(time_test, [y[0] for y in Y_test], 'b--', label='True Position')
axs[0].plot(time_test, [p[0] for p in test_predictions], 'r-', label='Predicted Position')
axs[0].set_title('Test Data - Position Over Time')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Position')
axs[0].legend()

# Plotting the true and predicted velocities
axs[1].plot(time_test, [y[1] for y in Y_test], 'b--', label='True Velocity')
axs[1].plot(time_test, [p[1] for p in test_predictions], 'r-', label='Predicted Velocity')
axs[1].set_title('Test Data - Velocity Over Time')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Velocity')
axs[1].legend()

plt.tight_layout()
plt.savefig('test_data_predictions_vs_true_values.png')  # Saving the figure as a PNG file
plt.close()  # Closing the figure to free up memory


In [149]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Data for plotting and animation
time = np.arange(len(predictions))
predictions = np.array(predictions)
true_states = sol.y[:, 10:110].T

# Setup the figure and subplots
fig, axs = plt.subplots(2, 1, figsize=(12, 10))
line1, = axs[0].plot([], [], 'r-', label='Predicted Position')
line2, = axs[0].plot([], [], 'b--', label='True Position')
line3, = axs[1].plot([], [], 'r-', label='Predicted Velocity')
line4, = axs[1].plot([], [], 'b--', label='True Velocity')

# Initializing the plot lines
def init():
    axs[0].set_xlim(0, 100)
    axs[0].set_ylim(np.min(true_states[:, 0]) - 0.5, np.max(true_states[:, 0]) + 0.5)
    axs[0].set_title('Position Over Time')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('Position')
    axs[0].legend()

    axs[1].set_xlim(0, 100)
    axs[1].set_ylim(np.min(true_states[:, 1]) - 0.5, np.max(true_states[:, 1]) + 0.5)
    axs[1].set_title('Velocity Over Time')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Velocity')
    axs[1].legend()

    return line1, line2, line3, line4

# Animation update function
def update(frame):
    line1.set_data(time[:frame], predictions[:frame, 0])
    line2.set_data(time[:frame], true_states[:frame, 0])
    line3.set_data(time[:frame], predictions[:frame, 1])
    line4.set_data(time[:frame], true_states[:frame, 1])
    return line1, line2, line3, line4

# Create and save animation
ani = FuncAnimation(fig, update, frames=len(time), init_func=init, blit=True)
ani.save('harmonic_oscillator.gif', fps=20, writer='imagemagick')

plt.close(fig)  # Close the figure to prevent it from displaying in a non-interactive environment


MovieWriter imagemagick unavailable; using Pillow instead.
